In [0]:
import urllib3
from bs4 import BeautifulSoup
urllib3.disable_warnings()
import regex
import json
import pandas as pd

In [0]:
media = 'https://www.aftonbladet.se/ledare'
http = urllib3.PoolManager()
r = http.request('GET', media)

In [0]:
soup = BeautifulSoup(r.data, 'html.parser')

In [0]:
ledare_start_page = soup.find_all('a', attrs={'class': '_2-jbn _35Mm_ _15oWD deH13 clearfix Xxu46'})

In [5]:
news_article_re = '/nyheter/'
arg_page_re = '\?page='
article_dict=dict()
df_output=pd.DataFrame()
for tag in ledare_start_page:
    ledare_author_name = tag.find('span', 
                                  attrs={'class': '_2zZZs abThemeColor'}).text
    if ledare_author_name == 'LEDARBLOGGEN':
      pass
    elif ledare_author_name in ['DANIEL SWEDIN','PERNILLA ERICSON','INGVAR PERSSON']:
      
      ledare_author_article = tag.get('href')
      r = http.request('GET', media+ledare_author_article)
      soup = BeautifulSoup(r.data, 'html.parser')
      ledare_personal_page = soup.find_all('h2', attrs={'class': '_1gz1t'})
      link = ledare_personal_page[0].find('a').get('href')
      ledare_personal_page_link = media.replace('/ledare','') + link
      
      count = 0
      while r.status == 200:
        arg = '?page='+str(count)
        
        r = http.request('GET', ledare_personal_page_link+arg)
        
        soup = BeautifulSoup(r.data, 'html.parser')
        
        ledare_page_articles = soup.find_all('div', attrs={'class': '_3p4DP'})
        for elem in ledare_page_articles:
          articles = elem.find_all('a')
          for link in articles:
            link_href = link.get('href')
            news_article_link = regex.match(news_article_re, link_href)
            arg_page_link = regex.match(arg_page_re, link_href)
            
            if news_article_link or arg_page_link:
              pass
            else:
              link_ledare_content = media.replace('/ledare','') + link_href
              r = http.request('GET', link_ledare_content)
              soup = BeautifulSoup(r.data, 'html.parser')
              ledare_content = soup.find_all('div', 
                                             attrs={'class': '_3p4DP _1lEgk'})

              header = ledare_content[0].find('h1', attrs={'class': '_11S-G'}).text
              try: 
                sub_header = ledare_content[0].find('h2', attrs={'class': '_3oEVr'}).text
              except: 
                sub_header=None
              text = ledare_content[0].find_all('p', attrs={'class': '_2ZkCB'})

              text_content = ''
              for text_elem in text:
                text_content += '\n' + text_elem.text

              article_dict = {'header': header,
                             'sub_header': sub_header,
                             'text_content': text_content, 
                             'author':ledare_author_name}
              df=pd.DataFrame.from_dict(article_dict,orient='index')
              df_output = pd.concat([df.T, df_output])
              
              
              break
        
        count+=1
        if count == 50:
          break
      if len(df_output)>=150: 
        break
          
      
df_output=df_output.reset_index(drop=True)
df_output.head(n=4)

,header,sub_header,text_content,author
0,Bowies konstskolahar stängt för gott,None,\nDaniel Swedin: Rockens kameleont gör sorti m...,DANIEL SWEDIN
1,Matfusket är en miljardaffär,None,\nDitt kravmärkta svenska kött kan i själva ve...,DANIEL SWEDIN
2,Lågavlönade ska packa Ica-kassen,None,\nDe enkla jobben är för få och lönerna är för...,DANIEL SWEDIN
3,Samhället måste våga möta hatet,None,\nEfter bombdådet i Oslo och massakern på Utøy...,DANIEL SWEDIN


In [0]:
df_output['author'].value_counts()

DANIEL SWEDIN       50
INGVAR PERSSON      50
PERNILLA ERICSON    40
Name: author, dtype: int64

In [0]:
df_output.to_csv('web_scarping_csv_140.csv')

In [0]:
from google.colab import files


files.download('web_scarping_csv_140.csv')